In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
from rdflib import Graph
import numpy as np

In [ ]:
# Load the RDF graph using rdflib
rdf_graph = Graph()
rdf_graph.parse("14_graph.nt", format="turtle")

print("Graph loaded")

In [ ]:
# Create node and edge lists
nodes = list(set([str(s) for s in rdf_graph.subjects()] + [str(o) for o in rdf_graph.objects()]))
node_idx = {node: i for i, node in enumerate(nodes)}

edges = [(node_idx[str(s)], node_idx[str(o)]) for s, p, o in rdf_graph if str(o) in node_idx]

# Convert edge list to torch tensors
edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()

# Generate random node features (for demonstration purposes)
num_nodes = len(nodes)
node_features = torch.rand((num_nodes, 10), dtype=torch.float)  # 10 features per node

# Create a Data object from torch_geometric
data = Data(x=node_features, edge_index=edge_index)

# Define a simple GCN model
class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return x

# Move model and data to MPS (Metal Performance Shaders) if available
device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')
data = data.to(device)
model = GCN(in_channels=10, hidden_channels=16, out_channels=10).to(device)

# Define an optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

# Training loop
model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = F.mse_loss(out, data.x)  # Using MSE loss for demonstration
    loss.backward()
    optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

print("Training finished")

In [ ]:
torch.save(model.state_dict(), 'models/gcn_embedding.pth')